# Run analysis on multiple files

In [22]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
import h5py

In [24]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-18'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_750'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FilamentLength_Sweeps/activity_time_750'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-28'

data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-10-18'

print(os.listdir(data_folder))

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.' and 'analysis' not in fileNames):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('No:of Simulation files: ', len(files_list))

def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)


    root_folder, file_name = os.path.split(file)

    # Create a data-packet to save for further analysis
    # Collect simulation metadata
#     metadata_path = os.path.join(root_folder, 'metadata.csv')
#     assert(os.path.exists(metadata_path))
#     df = pd.read_csv(metadata_path)
    
    # @@@ Better to use Metadata from the HDF5 file so there is no possible delinking of data and metadata
    df = pd.DataFrame({'N particles':[filament.Np],'radius':[filament.radius],'bond length':[filament.b0],
                       'spring constant':[filament.k], 'kappa_hat':[filament.kappa_hat], 
                       'force strength':[filament.F0], 'stresslet strength':[filament.S0], 
                       'potDipole strength':[filament.D0], 'simulation type':[filament.sim_type], 
                       'boundary condition 0':[filament.bc[0]], 'boundary condition 1':[filament.bc[-1]], 
                       ' activity time scale':[filament.activity_timescale], 'viscosity':[filament.mu]})
    
    
    periodic_flag = False 
    min_period = 0
    
    filament.filament_tip_coverage(save = True)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) == int(df[' activity time scale']*500)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period = filament.classify_filament_dynamics()
    else:
        simulation_completed = False

    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    df['period'] = min_period
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    
    
    
    
    # Calculate the filament tip angles (at constant phase) and save them to a file
    
    phase = 0
    angles = filament.compute_tip_angle_at_constant_phase(phase_value = phase)
    print(angles)
    
    

    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
    df.to_csv(os.path.join(save_folder, save_file))
    
    # Create a dataset and save the data
    save_file = file_name[:-5] + '_analysis.hdf5'


    with h5py.File(os.path.join(save_folder, save_file), "w") as f:
        dset = f.create_group("analysis data")
        dset.attrs['N particles'] = filament.Np
        dset.attrs['radius'] = filament.radius
        dset.attrs['bond length'] = filament.b0
        dset.attrs['spring constant'] = filament.k
        dset.attrs['kappa_hat'] = filament.kappa_hat
        dset.attrs['force strength'] = filament.F0
        dset.attrs['stresslet strength'] = filament.S0
        dset.attrs['potDipole strength'] = filament.D0
        dset.attrs['simulation type'] = filament.sim_type
        dset.attrs['activity time scale'] = filament.activity_timescale
        dset.attrs['viscosity'] = filament.mu
        dset.attrs['boundary condition 0'] = filament.bc[0]
        dset.attrs['boundary condition 1'] = filament.bc[-1]
        dset.attrs['Sim complete'] = simulation_completed
        
#         print(periodic_flag)
#         print(min_period)
#         dset.attrs['periodic'] = int(periodic_flag)
#         dset.attrs['period'] = int(min_period)
        
        dset.attrs['constant phase'] = phase
        dset.create_dataset("Tip angles", data = angles)
        
    
    

 79%|███████▉  | 95/120 [23:51:03<6:16:35, 903.83s/it]

['SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5256410256410255_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.9102564102564101_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.846153846153846_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_3.0_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_2.615384615384615_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_2.0384615384615383_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_0.9487179487179487_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.6538461538461537_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_2.1666666666666665_activityTime_7

In [25]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))




  0%|          | 0/160 [00:00<?, ?it/s][Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.3s


 15%|█▌        | 24/160 [00:01<00:07, 18.48it/s][Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    1.5s


 22%|██▎       | 36/160 [00:01<00:04, 26.08it/s][Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    1.7s


 30%|███       | 48/160 [00:01<00:03, 29.37it/s][Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.8s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    4.0s


 38%|███▊      | 60/160 [00:04<00:08, 11.53it/s][Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    4.2s


 45%|████▌     | 72/160 [00:04<00:05, 16.22it/s][Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    4.3s


 52%|█████▎    | 84/160 [00:04<00:03, 22.39it/s][Parallel(n_jobs=12)]: Batch computation too fast (0.1976s.) Setting batch_size=2.


 60%|██████    | 96/160 [00:05<00:04, 13.76it/s][Parallel(n_job

In [21]:
print(np.linspace(0.5,3,40))

[0.5        0.56410256 0.62820513 0.69230769 0.75641026 0.82051282
 0.88461538 0.94871795 1.01282051 1.07692308 1.14102564 1.20512821
 1.26923077 1.33333333 1.3974359  1.46153846 1.52564103 1.58974359
 1.65384615 1.71794872 1.78205128 1.84615385 1.91025641 1.97435897
 2.03846154 2.1025641  2.16666667 2.23076923 2.29487179 2.35897436
 2.42307692 2.48717949 2.55128205 2.61538462 2.67948718 2.74358974
 2.80769231 2.87179487 2.93589744 3.        ]
